In [1]:
import pandas as pd
import plotly.express as px
import dash
from dash import dcc, html
from dash.dependencies import Input, Output

In [2]:
df = pd.read_csv("data/immo_data_202208_v2_imputed.csv")

C:\Users\wartm\AppData\Local\Temp\ipykernel_8968\2881222879.py:1: DtypeWarning: Columns (74) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("data/immo_data_202208_v2_imputed.csv")


In [3]:
# Clean and prepare data
#df['price'] = df['CHF'].replace({'CHF ': '', '—': '', '\.—': '', ',': ''}, regex=True).astype(float)
df['Living space'] = df['Living space'].str.replace(' m²', '').astype(float)

In [4]:
df.head()

,Municipality,Living space,Availability,location,description,detailed_description,url,table,Municipality_merged,Living_space_merged,...,gde_workers_total,price_cleaned,type,Space extracted,rooms,plz_parsed,type_unified,Living_area_unified,provider,plz
0,Biberstein,100.0,On request,"5023 Biberstein, AG","3.5 rooms, 100 m²«Luxuriöse Attika-Wohnung mit...",DescriptionLuxuriöse Attika-Wohnung direkt an ...,https://www.immoscout24.ch//en/d/penthouse-buy...,b <article class=####Box-cYFBPY hKrxoH####><h2...,Biberstein,100 m²,...,331.0,1150000.0,penthouse,100.0,5.0,5023.0,penthouse,100.0,Immoscout24.ch,5023.0
1,Biberstein,156.0,On request,"Buhldenstrasse 8d5023 Biberstein, AG","4.5 rooms, 156 m²«Stilvolle Liegenschaft - ruh...",DescriptionStilvolle Liegenschaft an ruhiger L...,https://www.immoscout24.ch//en/d/terrace-house...,b <article class=####Box-cYFBPY hKrxoH####><h2...,Biberstein,156 m²,...,331.0,1420000.0,terrace-house,156.0,5.0,5023.0,terrace-house,156.0,Immoscout24.ch,5023.0
2,Crans-Montana,120.0,On request,"5022 Rombach, AG","2.5 rooms, 93 m²«Moderne, lichtdurchflutete At...","detail_responsive#description_title2,5 Zimmerw...",https://www.immoscout24.ch//en/d/penthouse-buy...,b <article class=####Box-cYFBPY hKrxoH####><h2...,Lugano,120 m²,...,33493.0,720000.0,penthouse,105.8,5.0,5022.0,penthouse,105.8,Immoscout24.ch,5022.0
3,Biberstein,154.0,On request,"Buhaldenstrasse 8A5023 Biberstein, AG","4.5 rooms, 154 m²«AgentSelly - Luxuriöses Eckh...",DescriptionDieses äusserst grosszügige Minergi...,https://www.immoscout24.ch//en/d/detached-hous...,b <article class=####Box-cYFBPY hKrxoH####><h2...,Biberstein,154 m²,...,331.0,1430000.0,detached-house,154.0,5.0,5023.0,detached-house,154.0,Immoscout24.ch,5023.0
4,Küttigen,142.0,On request,"5022 Rombach, AG","4.5 rooms, 142 m²«MIT GARTENSITZPLATZ UND VIEL...",DescriptionAus ehemals zwei Wohnungen wurde ei...,https://www.immoscout24.ch//en/d/flat-buy-romb...,b <article class=####Box-cYFBPY hKrxoH####><h2...,Küttigen,142 m²,...,1355.0,995000.0,flat,142.0,5.0,5022.0,flat,142.0,Immoscout24.ch,5022.0


In [5]:
df.isna().sum().sum()

np.int64(0)

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21466 entries, 0 to 21465
Data columns (total 80 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   Municipality                       21466 non-null  object 
 1   Living space                       21466 non-null  float64
 2   Availability                       21466 non-null  object 
 3   location                           21466 non-null  object 
 4   description                        21466 non-null  object 
 5   detailed_description               21466 non-null  object 
 6   url                                21466 non-null  object 
 7   table                              21466 non-null  object 
 8   Municipality_merged                21466 non-null  object 
 9   Living_space_merged                21466 non-null  object 
 10  Availability_merged                21466 non-null  object 
 11  location_parsed                    21466 non-null  obj

In [7]:
df = df[~df["price_cleaned"].isna()]

In [8]:
df= df.drop(columns=["price", "Living space"])

df= df.rename(columns={"price_cleaned": "price", "Living_space_merged": "Living_space"})


# LE 1
Performance testen mit Häuserpreise pro gemeinde oder alle Datenpunkte direkt laden.

In [ ]:
import numpy as np
import pandas as pd

multiplier = 100
n = df.shape[0]
augmented_list = [df]
numeric_cols = df.select_dtypes(include=[np.number]).columns

for i in range(multiplier - 1):
    df_syn = df.sample(n=n, replace=True, random_state=42 + i).reset_index(drop=True)
    for col in numeric_cols:
        # multiplikatives Rauschen
        factor = 1 + np.random.randn(n) * 0.01
        df_syn[col] = (df_syn[col] * factor).clip(lower=0)
    augmented_list.append(df_syn)

df_augmented = pd.concat(augmented_list, ignore_index=True)


In [ ]:
df_augmented.info()

In [ ]:
import dash
from dash import html, dcc
import dash_bootstrap_components as dbc
import plotly.express as px
import pandas as pd

# Use a Bootstrap theme
external_stylesheets = [dbc.themes.LUX]

# Karte vorbereiten (keine Filter/Slider mehr)
fig_map = px.scatter_mapbox(
    df,
    lat='lat',
    lon='lon',
    color='price',
    size='Living_area_unified',
    size_max=15,
    hover_name='Municipality',
    hover_data=['type_unified', 'price'],
    mapbox_style='carto-positron',
    zoom=6.5
)
fig_map.update_layout(margin={'r':0, 't':0, 'l':0, 'b':0})

app = dash.Dash(__name__, external_stylesheets=external_stylesheets)
server = app.server  # für Deployment

# Navbar
navbar = dbc.NavbarSimple(
    brand="🏠 Schweizer Immobilienpreis-Übersicht",
    color="primary",
    dark=True,
    fluid=True,
)


# Layout: nur Navbar, Karte, Footer
app.layout = dbc.Container(
    [
        navbar,
        # Vollbreite Karte ohne Controls
        dbc.Row(
            dbc.Col(dcc.Graph(id='map-overview', figure=fig_map), width=12),
            className="mb-4"
        )
    ],
    fluid=True,
    className="p-4"
)


In [ ]:
app.run(debug=True, port=8041)

In [35]:
import math
app = dash.Dash(__name__, external_stylesheets=[dbc.themes.LUX])
server = app.server  # Damit wir Flask-Routen definieren können

# -------------------------------
# 3) Hilfsfunktion: lat/lon → Pixelkoordinate im Tile
# -------------------------------
def latlon_to_pixel(lat: float, lon: float, z: int):
    """
    Liefert (px, py) die absoluten Pixelkoordinaten im Web-Mercator-Raster:
    px,py ∈ [0, 256*2^z)
    """
    lat_rad = math.radians(lat)
    n = 2 ** z
    px = (lon + 180.0) / 360.0 * 256 * n
    py = (
        (1.0 - math.log(math.tan(lat_rad) + (1 / math.cos(lat_rad))) / math.pi)
        / 2.0
        * 256
        * n
    )
    return px, py

# -------------------------------
# 4) Flask-Route: Tile-Server
# -------------------------------
@server.route("/tiles/<int:z>/<int:x>/<int:y>.png")
def serve_tile(z, x, y):
    """
    Gibt für z=0 und z=1 ein 256×256-PNG mit allen Punkten des DataFrames zurück,
    die in dieses Tile fallen. Für andere Zoom-Stufen: weiße Kachel.
    """
    size = (256, 256)
    background_color = (255, 255, 255)  # weiß, falls kein z=0 oder z=1

    img = Image.new("RGB", size, color=background_color)
    draw = ImageDraw.Draw(img)

    if z in (0, 1):
        # 1a) Punkte selektieren, die in dieses Tile fallen
        relevant_rows = []
        for idx, row in df.iterrows():
            lat = float(row["lat"])
            lon = float(row["lon"])
            px, py = latlon_to_pixel(lat, lon, z)
            if x * 256 <= px < (x + 1) * 256 and y * 256 <= py < (y + 1) * 256:
                rel_x = int(px - x * 256)
                rel_y = int(py - y * 256)
                relevant_rows.append((rel_x, rel_y, row))

        # 1b) Farbe festlegen, je nach Zoomstufe:
        point_color = (255, 0, 0) if z == 0 else (0, 0, 255)
        radius = 3

        for (rel_x, rel_y, row) in relevant_rows:
            draw.ellipse(
                [(rel_x - radius, rel_y - radius), (rel_x + radius, rel_y + radius)],
                fill=point_color,
                outline=None,
            )

        # 1c) Tile-Koordinaten und Anzahl in der Mitte zeichnen
        lines = [f"z={z}  x={x}  y={y}", f"Punkte: {len(relevant_rows)}"]
        try:
            font = ImageFont.truetype("arial.ttf", 14)
        except:
            font = ImageFont.load_default()

        # Jede Zeile einzeln messen
        max_w = 0
        total_h = 0
        line_heights = []
        for line in lines:
            w, h = draw.textsize(line, font=font)
            max_w = max(max_w, w)
            line_heights.append(h)
            total_h += h

        # Gesamtposition so berechnen, dass der Text zentriert ist
        x0 = (size[0] - max_w) / 2
        y0 = (size[1] - total_h) / 2

        # Zeilenweise zeichnen (mit kleinem Zeilenabstand)
        cur_y = y0
        for i, line in enumerate(lines):
            draw.text(
                (x0, cur_y),
                line,
                fill=(255, 255, 255),
                font=font,
            )
            cur_y += line_heights[i]

    # 1d) Bild in Bytes puffern und zurückgeben
    buf = io.BytesIO()
    img.save(buf, format="PNG")
    buf.seek(0)
    return send_file(buf, mimetype="image/png")

# -------------------------------
# 5) Plotly-Figur mit Mapbox + unserem Tile-Layer
# -------------------------------
fig = go.Figure(go.Scattermapbox())

fig.update_layout(
    mapbox={
        "style": "carto-positron",
        "zoom": 0,  # Start-Zoom 0 → zeigt rote Punkte
        "center": {"lat": df["lat"].mean(), "lon": df["lon"].mean()},
        "layers": [
            {
                "sourcetype": "raster",
                "source": ["http://localhost:8050/tiles/{z}/{x}/{y}.png"],
                "below": "traces",
                "opacity": 0.7,
            }
        ],
    },
    margin={"l": 0, "r": 0, "t": 0, "b": 0},
)

# -------------------------------
# 6) Navbar & Layout
# -------------------------------
navbar = dbc.NavbarSimple(
    brand="🔲 Experiment: Tiling mit eigenem Datensatz (z=0 rot, z=1 blau)",
    color="dark",
    dark=True,
    fluid=True,
)

app.layout = dbc.Container(
    [
        navbar,
        dbc.Row(
            dbc.Col(dcc.Graph(id="map-tiles-demo", figure=fig), width=12),
            className="mt-3",
        ),
        dbc.Row(
            dbc.Col(
                dbc.Alert(
                    "Teste Zoomstufe 0 und 1: Bei z=0 siehst du alle Punkte rot in der einzigen Kachel. "
                    "Bei z=1 werden die Punkte in vier blaue Kacheln aufgeteilt (x/y=0 oder 1). "
                    "Andere Zoomstufen sind weiß.",
                    color="info",
                ),
                width=12,
            ),
            className="mt-2",
        ),
    ],
    fluid=True,
    className="p-3",
)

# -------------------------------
# 7) Server starten
# -------------------------------
if __name__ == "__main__":
    app.run(debug=True)

In [ ]:
import time
from dash import html, dcc, dash_table, Input, Output, State, callback
import dash_bootstrap_components as dbc
import matplotlib.pyplot as plt

# Define fractions of data to test (e.g., 10%, 50%, 100%)
fractions = [0.1,0.25, 0.5,0.75, 1.0]
repeats = 10
perf_times = {frac: [] for frac in fractions}
map_figs = {}

# Measure startup time for each fraction, repeated trials
for frac in fractions:
    for i in range(repeats):
        t0 = time.perf_counter()
        df_sub = df_augmented.sample(frac=frac, random_state=42)
        fig_map = px.scatter_mapbox(
            df_sub,
            lat='lat', lon='lon',
            color='price', size='Living_area_unified', size_max=15,
            hover_name='Municipality', hover_data=['type_unified', 'price'],
            mapbox_style='carto-positron', zoom=6
        )
        elapsed = time.perf_counter() - t0
        perf_times[frac].append(elapsed)
    # store the full-data map from the first repeat
    if frac == 1.0:
        map_figs['full'] = fig_map

# Compute means and standard deviations
means = [sum(perf_times[frac]) / repeats for frac in fractions]
stds = [(pd.Series(perf_times[frac]).std()) for frac in fractions]
percentages = [int(f * 100) for f in fractions]

# Plot performance with error bars using Matplotlib
plt.figure(figsize=(8, 5))
plt.errorbar(percentages, means, yerr=stds, marker='o', linestyle='-')
plt.title('Map Startup Time vs Data Fraction')
plt.xlabel('Data Size (%)')
plt.ylabel('Startup Time (seconds)')
plt.grid(True)
plt.tight_layout()
plt.show()

# Initialize Dash app for map visualization only
app = Dash(__name__)

# Build layout with just the full-data map
ing_app = html.Div([
    html.H1('Swiss Real Estate Map Performance'),
    dcc.Graph(figure=map_figs['full'])
], style={'width': '90%', 'margin': '0 auto'})

app.layout = ing_app


In [ ]:
app.run(debug=True, port=8049)

## LOD

In [ ]:
df_augmented.info()

In [ ]:
import time
import geopandas as gpd
import json
from dash import Dash, dcc, html
import plotly.express as px


# Geo-Daten einlesen und Vorverarbeiten ---
gdf = gpd.read_file(
    "swissBOUNDARIES3D_1_5_LV95_LN02.gpkg",
    layer="tlm_hoheitsgebiet"
)
gdf = gdf.to_crs(epsg=4326)

# 1.1 Durchschnittspreis pro Gemeinde berechnen
start_fp = time.time()
avg_price = (
    df_augmented.groupby("Municipality", as_index=False)["price"]
      .mean()
      .rename(columns={"price": "avg_price"})
)

gdf_merged = gdf.merge(
    avg_price,
    left_on="name",
    right_on="Municipality",
    how="left"
)
gdf_simple = gdf_merged[["name", "avg_price", "geometry"]]
geojson = json.loads(gdf_simple.to_json())
end_fp = time.time()
print(f"[Choropleth] Vorbereitung (inkl. Merge) dauerte: {end_fp - start_fp:.2f} s")

# Choropleth-Figur erstellen ---
t1 = time.time()
fig_choro = px.choropleth_mapbox(
    gdf_simple,
    geojson=geojson,
    locations="name",
    featureidkey="properties.name",
    color="avg_price",
    mapbox_style="open-street-map",
    zoom=8,
    center={"lat": 46.8, "lon": 8.2},
    opacity=0.6,
    color_continuous_scale="Viridis",
    labels={"avg_price": "Durchschnittspreis (CHF)"}
)
t2 = time.time()
print(f"[Choropleth] Figure-Erzeugung: {t2 - t1:.2f} s")

# --- 3) Dash-App definieren und starten ---
app = Dash(__name__)
app.layout = html.Div([
    html.H1("Durchschnittspreis pro Gemeinde (CH)"),
    dcc.Graph(id="price-map", figure=fig_choro)
])

app.run(debug=True, port=8040)


In [ ]:

# ----------------------------------
# SCATTER-FIGUR ERSTELLEN
# ----------------------------------
t0 = time.time()
fig_scatter = px.scatter_mapbox(
    df_augmented,
    lat="lat",
    lon="lon",
    color="price",           # Farbskala nach Preis
    mapbox_style="open-street-map",
    zoom=8,
    center={"lat": 46.8, "lon": 8.2},
    opacity=0.6,
    labels={"price": "Preis (CHF)"}
)
t1 = time.time()
print(f"[Scatter] Figure-Erzeugung: {t1 - t0:.2f} Sekunden")

app = Dash(__name__)
app.layout = html.Div([
    html.H1("Alle Transaktionen als Punkte"),
    dcc.Graph(id="scatter-map", figure=fig_scatter)
])

#app.run(debug=True, port=8051)


# LE 2

In [ ]:
import pandas as pd
import plotly.express as px
from dash import Dash, dcc, html, Input, Output, dash_table
import numpy as np

In [21]:
# Initialize Dash
app = Dash(__name__)
min_price = int(df['price_cleaned'].min())
max_price = int(df['price_cleaned'].max())

marks = {
    int(price): f"{int(price)//1000}K"
    for price in np.arange(min_price, max_price + 1, 2_500_000)
}

# App Layout
app.layout = html.Div([
    html.H1("Swiss Real Estate Dashboard"),

    html.Div([
        html.Div([
            html.Label("Municipality"),
            dcc.Dropdown(
                id='municipality-dropdown',
                options=[{'label': mun, 'value': mun} for mun in sorted(df['Municipality'].dropna().unique())],
                multi=True,
                placeholder='Select Municipality'
            ),
            html.Label("Property Type"),
            dcc.Dropdown(
                id='type-dropdown',
                options=[{'label': t, 'value': t} for t in sorted(df['type_unified'].dropna().unique())],
                multi=True,
                placeholder='Select Property Type'
            ),
            html.Label("Price Range (CHF)"),


            dcc.RangeSlider(
                id='price-slider',
                min=min_price,
                max=max_price,
                step=50_000,
                value=[min_price, max_price],
                marks=marks
            ),
            html.Div(id='price-range', style={'marginTop': 10})
        ], style={'width': '30%', 'padding': 10}),

        html.Div([
            dcc.Graph(id='map-overview'),
            dcc.Graph(id='scatter-overview')
        ], style={'width': '70%', 'padding': 10}),

    ], style={'display': 'flex', 'flex-direction': 'row'}),

    html.Hr(),

    html.Div([
        html.H3("Property Details"),
        dash_table.DataTable(
            id='details-table',
            columns=[{"name": c, "id": c} for c in ['Municipality', 'Living space', 'price_cleaned', 'description', 'url']],
            page_size=5,
            style_table={'overflowX': 'auto'},
        )
    ], style={'padding': 10})
])

# Callbacks
@app.callback(
    [Output('map-overview', 'figure'),
     Output('scatter-overview', 'figure'),
     Output('price-range', 'children')],
    [Input('municipality-dropdown', 'value'),
     Input('type-dropdown', 'value'),
     Input('price-slider', 'value')]
)
def update_graph(municipalities, types, price_range):
    # Filter by price range
    filtered_df = df[(df['price_cleaned'] >= price_range[0]) & (df['price_cleaned'] <= price_range[1])]

    # Filter by municipality
    if municipalities:
        filtered_df = filtered_df[filtered_df['Municipality'].isin(municipalities)]
    # Filter by property type
    if types:
        filtered_df = filtered_df[filtered_df['type_unified'].isin(types)]

    # Map overview
    fig_map = px.scatter_mapbox(
        filtered_df,
        lat='lat',
        lon='lon',
        color='price_cleaned',
        size='Living space',
        hover_name='Municipality',
        hover_data={'price_cleaned': True, 'type_unified': True},
        mapbox_style='open-street-map',
        zoom=8
    )

    # Scatter overview
    fig_scatter = px.scatter(
        filtered_df,
        x='Living space',
        y='price_cleaned',
        color='Municipality',
        hover_data=['description']
    )

    price_text = f"Selected Price Range: CHF {price_range[0]:,.0f} - CHF {price_range[1]:,.0f}"

    return fig_map, fig_scatter, price_text

@app.callback(
    Output('details-table', 'data'),
    Input('scatter-overview', 'clickData')
)
def display_details(clickData):
    if clickData:
        x_val = clickData['points'][0]['x']
        y_val = clickData['points'][0]['y']
        selected_df = df[(df['Living space'] == x_val) & (df['price_cleaned'] == y_val)]
        return selected_df[['Municipality', 'Living space', 'price_cleaned', 'description', 'url']].to_dict('records')
    return []


NameError: name 'Dash' is not defined

In [22]:
#app.run(debug=True, port=8050)

In [ ]:
import dash
from dash import html, dcc, dash_table, Input, Output
import dash_bootstrap_components as dbc
import plotly.express as px

# Use a Bootstrap theme
external_stylesheets = [dbc.themes.LUX]

step = 250_000

# floor the min down to the nearest step
raw_min = df['price'].min()
slider_min = int((raw_min // step) * step)

# ceil the max up to the nearest step
raw_max = df['price'].max()
slider_max = int(np.ceil(raw_max / step) * step)


app = dash.Dash(__name__, external_stylesheets=external_stylesheets)
server = app.server  # for deployment

# FilterControls card
filter_card = dbc.Card(
    [
        dbc.CardHeader(html.H5("Filters")),
        dbc.CardBody(
            [
                html.Div(
                    [
                        dbc.Label("Municipality", className="form-label"),
                        dcc.Dropdown(
                            id='municipality-dropdown',
                            options=[{'label': m, 'value': m} for m in sorted(df['Municipality'].dropna().unique())],
                            multi=True,
                            placeholder='Select…'
                        ),
                    ],
                    className="mb-3"
                ),
                html.Div(
                    [
                        dbc.Label("Property Type", className="form-label"),
                        dcc.Dropdown(
                            id='type-dropdown',
                            options=[{'label': t, 'value': t} for t in sorted(df['type_unified'].dropna().unique())],
                            multi=True,
                            placeholder='Select…'
                        ),
                    ],
                    className="mb-3"
                ),
                html.Div(
                    [
                        dbc.Label("Price Range (CHF)", className="form-label"),
                        dcc.RangeSlider(
                            id='price-slider',
                            min=slider_min,
                            max=slider_max,
                            step=step,
                            value=[slider_min, slider_max],
                            marks={
                                p: f"{p//1_000_000}M"  # or f"{p/1000:.0f}k"
                                for p in range(slider_min, slider_max + 1, 2_000_000)
                            },
                            tooltip={"placement": "bottom", "always_visible": False},
                        ),
                        html.Div(id='price-range', className='mt-2 text-muted')
                    ],
                    className="mb-3"
                ),
            ]
        ),
    ],
    className="mb-4"
)

# Navbar
navbar = dbc.NavbarSimple(
    brand="🏠 Swiss Real Estate Dashboard",
    color="primary",
    dark=True,
    fluid=True,
)

# Footer
footer = dbc.Container(
    html.Footer("2025 Swiss Real Estate Insights", className="text-center text-muted py-3"),
    fluid=True,
)

app.layout = dbc.Container(
    [
        navbar,
        filter_card,
        dbc.Row(
            [
                dbc.Col(dcc.Graph(id='map-overview'), md=6),
                dbc.Col(dcc.Graph(id='scatter-overview'), md=6),
            ],
            className="mb-4",
        ),
        html.Hr(),
        html.H4("Property Details", className="mt-4"),
        dash_table.DataTable(
            id='details-table',
            columns=[{"name": c, "id": c} for c in
                     ['Municipality', 'Living space', 'price', 'description', 'url']],
            page_size=5,
            style_table={'overflowX': 'auto'},
            style_header={
                'backgroundColor': '#f8f9fa',
                'fontWeight': 'bold'
            },
            style_cell={'padding': '8px', 'textAlign': 'left'},
            row_selectable='single',
            cell_selectable=False,
        ),
        footer
    ],
    fluid=True,
    className="p-4"
)

# Callbacks
@app.callback(
    [Output('map-overview', 'figure'),
     Output('scatter-overview', 'figure'),
     Output('price-range', 'children')],
    [Input('municipality-dropdown', 'value'),
     Input('type-dropdown', 'value'),
     Input('price-slider', 'value')]
)
def update_graph(municipalities, types, price_range):
    dff = df[(df.price.between(price_range[0], price_range[1]))]
    if municipalities:
        dff = dff[dff.Municipality.isin(municipalities)]
    if types:
        dff = dff[dff.type_unified.isin(types)]

    fig_map = px.scatter_mapbox(
        dff, lat='lat', lon='lon',
        color='price', size='Living space',
        hover_name='Municipality', hover_data=['type_unified', 'price'],
        mapbox_style='carto-positron', zoom=8
    )
    fig_map.update_layout(margin={'r':0,'t':0,'l':0,'b':0})

    fig_scatter = px.scatter(
        dff, x='Living space', y='price',
        color='Municipality', hover_data=['description']
    )
    fig_scatter.update_traces(marker=dict(opacity=0.7, line=dict(width=0.5)))

    price_text = f"Selected Price Range: CHF {price_range[0]:,} – {price_range[1]:,}"
    return fig_map, fig_scatter, price_text

@app.callback(
    Output('details-table', 'data'),
    Input('scatter-overview', 'clickData')
)
def display_details(clickData):
    if not clickData:
        return []
    x, y = clickData['points'][0]['x'], clickData['points'][0]['y']
    sel = df[(df['Living space']==x) & (df['price']==y)]
    return sel[['Municipality','Living space','price','description','url']].to_dict('records')

In [ ]:
#app.run(debug=True, port=8051)


In [ ]:
import dash
from dash import html, dcc, dash_table, Input, Output
import dash_bootstrap_components as dbc
import plotly.express as px
import numpy as np

# Use a Bootstrap theme
external_stylesheets = [dbc.themes.LUX]

# Slider step size
step = 250_000

# Ensure df has numeric columns 'price' and 'Living_area_unified'
raw_min = df['price'].min()
slider_min = int((raw_min // step) * step)
raw_max = df['price'].max()
slider_max = int(np.ceil(raw_max / step) * step)

app = dash.Dash(__name__, external_stylesheets=external_stylesheets)
server = app.server  # for deployment

# Filter controls card
filter_card = dbc.Card(
    [
        dbc.CardHeader(html.H5("Filters")),
        dbc.CardBody(
            [
                dbc.Label("Municipality", html_for='municipality-dropdown'),
                dcc.Dropdown(
                    id='municipality-dropdown',
                    options=[{'label': m, 'value': m} for m in sorted(df['Municipality'].dropna().unique())],
                    multi=True,
                    placeholder='Select…'
                ),
                html.Br(),
                dbc.Label("Property Type", html_for='type-dropdown'),
                dcc.Dropdown(
                    id='type-dropdown',
                    options=[{'label': t, 'value': t} for t in sorted(df['type_unified'].dropna().unique())],
                    multi=True,
                    placeholder='Select…'
                ),
                html.Br(),
                dbc.Label("Price Range (CHF)", html_for='price-slider'),
                dcc.RangeSlider(
                    id='price-slider',
                    min=slider_min,
                    max=slider_max,
                    step=step,
                    value=[slider_min, slider_max],
                    marks={p: f"{p//1_000_000}M" for p in range(slider_min, slider_max + 1, 2_000_000)},
                    tooltip={"placement": "bottom", "always_visible": False},
                ),
                html.Div(id='price-range', className='mt-2 text-muted')
            ]
        ),
    ],
    className="mb-4"
)

# Navbar
navbar = dbc.NavbarSimple(
    brand="🏠 Swiss Real Estate Dashboard",
    color="primary",
    dark=True,
    fluid=True,
)

# Footer
footer = dbc.Container(
    html.Footer("2025 Swiss Real Estate Insights", className="text-center text-muted py-3"),
    fluid=True,
)

# Layout: map on top, histogram & scatter below
app.layout = dbc.Container(
    [
        navbar,
        filter_card,
        # Full-width map
        dbc.Row(
            dbc.Col(dcc.Graph(id='map-overview'), width=12),
            className="mb-4"
        ),
        # Two-column row: histogram and scatter
        dbc.Row(
            [
                dbc.Col(dcc.Graph(id='histogram-price'), md=6),
                dbc.Col(dcc.Graph(id='scatter-overview'), md=6),
            ],
            className="mb-4"
        ),
        html.Hr(),
        html.H4("Property Details", className="mt-4"),
        dash_table.DataTable(
            id='details-table',
            columns=[
                {"name": "Municipality", "id": "Municipality"},
                {"name": "Living area (m²)", "id": "Living_area_unified"},
                {"name": "Price (CHF)", "id": "price"},
                {"name": "Description", "id": "description"},
                {"name": "URL", "id": "url"}
            ],
            page_size=5,
            style_table={'overflowX': 'auto'},
            style_header={'backgroundColor': '#f8f9fa', 'fontWeight': 'bold'},
            style_cell={'padding': '8px', 'textAlign': 'left'},
            row_selectable='single',
            cell_selectable=False
        ),
        footer
    ],
    fluid=True,
    className="p-4"
)

# Callbacks to filter data

def filter_df(municipalities, types, price_range):
    dff = df[df['price'].between(price_range[0], price_range[1])]
    if municipalities:
        dff = dff[dff['Municipality'].isin(municipalities)]
    if types:
        dff = dff[dff['type_unified'].isin(types)]
    return dff

@app.callback(
    [Output('map-overview', 'figure'),
     Output('histogram-price', 'figure'),
     Output('scatter-overview', 'figure'),
     Output('price-range', 'children')],
    [Input('municipality-dropdown', 'value'),
     Input('type-dropdown', 'value'),
     Input('price-slider', 'value')]
)
def update_graph(municipalities, types, price_range):
    dff = filter_df(municipalities, types, price_range)

    # Map
    fig_map = px.scatter_mapbox(
        dff,
        lat='lat', lon='lon',
        color='price', size='Living_area_unified', size_max=15,
        hover_name='Municipality', hover_data=['type_unified', 'price'],
        mapbox_style='carto-positron', zoom=8
    )
    fig_map.update_layout(margin={'r':0,'t':0,'l':0,'b':0})

    # Histogram
    bins_count = int((slider_max - slider_min) / step)
    fig_hist = px.histogram(
        dff,
        x='price',
        title='Price Distribution (CHF)'
    )
    fig_hist.update_layout(margin={'r':0,'t':30,'l':0,'b':0})

    # Scatter
    fig_scatter = px.scatter(
        dff,
        x='Living_area_unified', y='price',
        color='Municipality', hover_data=['description']
    )
    fig_scatter.update_traces(marker=dict(opacity=0.7, line=dict(width=0.5)))
    fig_scatter.update_layout(
        margin={'r':0,'t':0,'l':0,'b':0},
        xaxis_title='Living area (m²)',
        yaxis_title='Price (CHF)'
    )

    price_text = f"Selected Price Range: CHF {price_range[0]:,} – {price_range[1]:,}"
    return fig_map, fig_hist, fig_scatter, price_text

@app.callback(
    Output('details-table', 'data'),
    Input('scatter-overview', 'clickData')
)
def display_details(clickData):
    if not clickData:
        return []
    x = clickData['points'][0]['x']
    y = clickData['points'][0]['y']
    sel = df[(df['Living_area_unified']==x) & (df['price']==y)]
    return sel[['Municipality', 'Living_area_unified', 'price', 'description', 'url']].to_dict('records')

In [ ]:
app.run(debug=True, port=8052)

# LE 3

In [25]:
import dash
from dash import html, dcc, dash_table, Input, Output, State
import dash_bootstrap_components as dbc
import plotly.express as px
import numpy as np

# Assume `df` is your loaded DataFrame with columns:
# 'price', 'lat', 'lon', 'Municipality', 'type_unified', 'Living_area_unified', 'description', 'url'

# Use a Bootstrap theme
external_stylesheets = [dbc.themes.LUX]

# Slider step size
def init_slider(df, step=250_000):
    raw_min = df['price'].min()
    slider_min = int((raw_min // step) * step)
    raw_max = df['price'].max()
    slider_max = int(np.ceil(raw_max / step) * step)
    return slider_min, slider_max

slider_min, slider_max = init_slider(df)
step = 250_000

app = dash.Dash(__name__, external_stylesheets=external_stylesheets)
server = app.server  # for deployment

# Store for Undo/Redo history
history_store = dcc.Store(
    id='history',
    data={
        'past': [],
        'present': {
            'municipality': [],
            'types': [],
            'price': [slider_min, slider_max]
        },
        'future': []
    }
)

# Breadcrumb nav
breadcrumbs = html.Nav(id='breadcrumbs', className='breadcrumb mb-3')

# Undo/Redo buttons
undo_redo = html.Div([
    dbc.Button("Undo", id='btn-undo', color='secondary', size='sm', className='me-2'),
    dbc.Button("Redo", id='btn-redo', color='secondary', size='sm')
], className='mb-3')

# Filter controls card
filter_card = dbc.Card([
    dbc.CardHeader(html.H5("Filters")),
    dbc.CardBody([
        dbc.Label("Municipality", html_for='municipality-dropdown'),
        dcc.Dropdown(id='municipality-dropdown', multi=True, placeholder='Select…'),
        html.Br(),
        dbc.Label("Property Type", html_for='type-dropdown'),
        dcc.Dropdown(id='type-dropdown', multi=True, placeholder='Select…'),
        html.Br(),
        dbc.Label("Price Range (CHF)", html_for='price-slider'),
        dcc.RangeSlider(
            id='price-slider',
            min=slider_min,
            max=slider_max,
            step=step,
            value=[slider_min, slider_max],
            marks={p: f"{p//1_000_000}M" for p in range(slider_min, slider_max+1, 2_000_000)},
            tooltip={"placement": "bottom", "always_visible": False}
        ),
        html.Div(id='price-range', className='mt-2 text-muted')
    ])
], className="mb-4")

# Navbar and Footer
navbar = dbc.NavbarSimple(
    brand="🏠 Swiss Real Estate Dashboard",
    color="primary", dark=True, fluid=True
)
footer = dbc.Container(
    html.Footer("2025 Swiss Real Estate Insights", className="text-center text-muted py-3"),
    fluid=True
)

# App layout
def serve_layout():
    return dbc.Container([
        dcc.Location(id='url', refresh=False),
        breadcrumbs,
        navbar,
        history_store,
        undo_redo,
        filter_card,
        dbc.Row(dbc.Col(dcc.Graph(id='map-overview'), width=12), className="mb-4"),
        dbc.Row([
            dbc.Col(dcc.Graph(id='histogram-price'), md=6),
            dbc.Col(dcc.Graph(id='scatter-overview'), md=6)
        ], className="mb-4"),
        html.Hr(),
        html.H4("Property Details", className="mt-4"),
        dash_table.DataTable(
            id='details-table',
            page_size=5,
            style_table={'overflowX': 'auto'},
            style_header={'backgroundColor': '#f8f9fa', 'fontWeight': 'bold'},
            style_cell={'padding': '8px', 'textAlign': 'left'},
            row_selectable='single',
            cell_selectable=False,
            columns=[
                {"name": "Municipality",       "id": "Municipality"},
                {"name": "Living area (m²)",   "id": "Living_area_unified"},
                {"name": "Price (CHF)",        "id": "price"},
                {"name": "Description",        "id": "description"},
                {"name": "URL",                "id": "url"}
            ]
        ),
        footer
    ], fluid=True, className="p-4")

app.layout = serve_layout

# ---------- Callbacks ----------

# Breadcrumbs
@app.callback(
    Output('breadcrumbs', 'children'),
    Input('url', 'pathname')
)
def update_breadcrumbs(pathname):
    segments = [seg for seg in pathname.split('/') if seg]
    crumbs = []
    for i, seg in enumerate(segments):
        link = '/' + '/'.join(segments[:i+1])
        crumbs.append(html.A(seg.capitalize(), href=link, className='breadcrumb-item'))
        if i < len(segments)-1:
            crumbs.append(html.Span('/', className='mx-1 text-muted'))
    return crumbs

# Unified filter + undo/redo callback
@app.callback(
    Output('municipality-dropdown', 'value'),
    Output('type-dropdown',        'value'),
    Output('price-slider',        'value'),
    Output('history',             'data'),
    Input('municipality-dropdown','value'),
    Input('type-dropdown',       'value'),
    Input('price-slider',        'value'),
    Input('btn-undo',            'n_clicks'),
    Input('btn-redo',            'n_clicks'),
    State('history',             'data'),
    prevent_initial_call=True
)
def sync_history_and_filters(mun, types, price, undo_n, redo_n, hist):
    ctx = dash.callback_context
    if not ctx.triggered:
        raise dash.exceptions.PreventUpdate
    trigger = ctx.triggered[0]['prop_id'].split('.')[0]

    past    = hist['past']
    present = hist['present']
    future  = hist['future']

    if trigger == 'btn-undo':
        if past:
            prev = past.pop()
            future.insert(0, present)
            new_state = prev
        else:
            new_state = present
    elif trigger == 'btn-redo':
        if future:
            nxt = future.pop(0)
            past.append(present)
            new_state = nxt
        else:
            new_state = present
    else:
        changed_state = {
            'municipality': mun   or [],
            'types':        types or [],
            'price':        price
        }
        if changed_state != present:
            past.append(present)
            past = past[-50:]
            future = []
            new_state = changed_state
        else:
            new_state = present

    updated_history = {
        'past':    past,
        'present': new_state,
        'future':  future
    }

    return (
        new_state['municipality'],
        new_state['types'],
        new_state['price'],
        updated_history
    )

# Filter helper
def filter_df(municipalities, types, price_range):
    dff = df[df['price'].between(price_range[0], price_range[1])]
    if municipalities:
        dff = dff[dff['Municipality'].isin(municipalities)]
    if types:
        dff = dff[dff['type_unified'].isin(types)]
    return dff

# Main graph update
@app.callback(
    Output('map-overview', 'figure'),
    Output('histogram-price', 'figure'),
    Output('scatter-overview', 'figure'),
    Output('price-range', 'children'),
    Input('municipality-dropdown', 'value'),
    Input('type-dropdown',         'value'),
    Input('price-slider',         'value')
)
def update_graph(municipalities, types, price_range):
    dff = filter_df(municipalities, types, price_range)

    # Map
    fig_map = px.scatter_mapbox(
        dff, lat='lat', lon='lon', color='price',
        size='Living_area_unified', size_max=15,
        hover_name='Municipality',
        hover_data=['type_unified', 'price'],
        mapbox_style='carto-positron',
        zoom=8
    )
    fig_map.update_layout(
        margin={'r':0,'t':0,'l':0,'b':0},
        transition={'duration': 300}
    )

    # Histogram
    fig_hist = px.histogram(dff, x='price', title='Price Distribution (CHF)')
    fig_hist.update_layout(margin={'r':0,'t':30,'l':0,'b':0})

    # Scatter
    fig_scatter = px.scatter(
        dff, x='Living_area_unified', y='price',
        color='Municipality', hover_data=['description']
    )
    fig_scatter.update_traces(marker=dict(opacity=0.7, line=dict(width=0.5)))
    fig_scatter.update_layout(
        margin={'r':0,'t':0,'l':0,'b':0},
        xaxis_title='Living area (m²)',
        yaxis_title='Price (CHF)'
    )

    price_text = f"Selected Price Range: CHF {price_range[0]:,} – {price_range[1]:,}"
    return fig_map, fig_hist, fig_scatter, price_text

# Details table
@app.callback(
    Output('details-table', 'data'),
    Input('scatter-overview', 'clickData')
)
def display_details(clickData):
    if not clickData:
        return []
    x = clickData['points'][0]['x']
    y = clickData['points'][0]['y']
    sel = df[(df['Living_area_unified'] == x) & (df['price'] == y)]
    return sel[['Municipality', 'Living_area_unified', 'price', 'description', 'url']].to_dict('records')


In [26]:
app.run(debug=True,port=8053)
